In [9]:
#pip install rasterstats

In [5]:
#pip install matplotlib

In [1]:
#pip install scikit-learn


In [16]:
import pandas as pd
import rasterio
from rasterstats import zonal_stats
from osgeo import gdal
import geopandas as gpd
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np
from shapely.geometry import Point
from shapely.geometry import Polygon
from pyproj import CRS
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import pyproj

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [3]:
landcover = pd.read_csv("C:/Users/omhai/OneDrive/Desktop/Shetty/Capstone Project/Landcover/Isaias_Nj_Lat_Long_500.csv")
landcover.head()

,Unnamed: 0,ID,latitude,longitude,value
0,0,0,41.415450,-75.846241,128
1,1,1,41.410565,-75.846241,128
2,2,2,41.405680,-75.846241,128
3,3,3,41.400794,-75.846241,128
4,4,4,41.395909,-75.846241,128


In [4]:
wind = pd.read_csv("C:/Users/omhai/OneDrive/Desktop/Shetty/Capstone Project/Isaias_NJ_speed_backwinds_akshay.csv")
wind.head()

,ID,Unnamed: 0,latitude,longitude,value,Distance,Vmax,Rmax
0,0,0,41.415450,-75.846241,128,325.294594,30.030877,120.546311
1,1,1,41.410565,-75.846241,128,325.583200,30.100979,120.546311
2,2,2,41.405680,-75.846241,128,325.871899,30.102396,120.546311
3,3,3,41.400794,-75.846241,128,326.160691,30.173096,120.546311
4,4,4,41.395909,-75.846241,128,326.449577,30.174527,120.546311


In [5]:
print(landcover.shape)
print(wind.shape)

(131364, 5)
(131364, 8)


- get zonal histogram for landcover data 

In [6]:
#check resolution of raster
with rasterio.open("C:/Users/omhai/OneDrive/Desktop/Shetty/Capstone Project/Landcover/NJ_All Layers_4326.tif") as src:
    # Print the bounding box coordinates
    print('Bounding box coordinates:', src.bounds)

    # Print the CRS
    print('CRS:', src.crs)

    #print resolution of raster
    print('Resolution:', src.res)

Bounding box coordinates: BoundingBox(left=-75.84624120900405, bottom=38.81163163562519, right=-73.54287375628081, top=41.41545006869191)
CRS: EPSG:4326
Resolution: (0.00027018973052471917, 0.00027018973052471917)


In [7]:
landcover.head()

,Unnamed: 0,ID,latitude,longitude,value
0,0,0,41.415450,-75.846241,128
1,1,1,41.410565,-75.846241,128
2,2,2,41.405680,-75.846241,128
3,3,3,41.400794,-75.846241,128
4,4,4,41.395909,-75.846241,128


In [12]:
landcover[["latitude","longitude"]].head()

,latitude,longitude
0,41.415450,-75.846241
1,41.410565,-75.846241
2,41.405680,-75.846241
3,41.400794,-75.846241
4,41.395909,-75.846241


In [40]:
landcover_zonal = pd.read_csv("C:/Users/omhai/OneDrive/Desktop/Shetty/Capstone Project/Landcover/zonal_500.csv")
landcover_zonal.head()

,id,left,top,right,bottom,HISTO_0,HISTO_1,HISTO_5,HISTO_6,HISTO_8,HISTO_10,HISTO_14,HISTO_15,HISTO_16,HISTO_17,HISTO_18,HISTO_NODATA
0,1,2054340,-81750,2054840,-82250,0,0,0,0,0,0,0,0,0,0,0,289
1,2,2054340,-82250,2054840,-82750,0,0,0,0,0,0,0,0,0,0,0,272
2,3,2054340,-82750,2054840,-83250,0,0,0,0,0,0,0,0,0,0,0,289
3,4,2054340,-83250,2054840,-83750,0,0,0,0,0,0,0,0,0,0,0,289
4,5,2054340,-83750,2054840,-84250,0,0,0,0,0,0,0,0,0,0,0,272


In [41]:
histo_columns = ['HISTO_0', 'HISTO_1', 'HISTO_5', 'HISTO_6', 'HISTO_8', 'HISTO_10',
                 'HISTO_14', 'HISTO_15', 'HISTO_16', 'HISTO_17', 'HISTO_18', 'HISTO_NODATA']

# Convert HISTO columns to percentages
landcover_zonal[histo_columns] = landcover_zonal[histo_columns].div(landcover_zonal[histo_columns].sum(axis=1), axis=0) * 100
landcover_zonal.head()

,id,left,top,right,bottom,HISTO_0,HISTO_1,HISTO_5,HISTO_6,HISTO_8,HISTO_10,HISTO_14,HISTO_15,HISTO_16,HISTO_17,HISTO_18,HISTO_NODATA
0,1,2054340,-81750,2054840,-82250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0
1,2,2054340,-82250,2054840,-82750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0
2,3,2054340,-82750,2054840,-83250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0
3,4,2054340,-83250,2054840,-83750,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0
4,5,2054340,-83750,2054840,-84250,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0


#### Get Coordinates in degree from polygon grid

In [30]:
grid_latlong = gpd.read_file("C:/Users/omhai/OneDrive/Desktop/Shetty/Capstone Project/Landcover/Temp/Grid_Azi_500.shp")
grid_latlong.head()

,id,left,top,right,bottom,geometry
0,1.0,2054340.0,-81750.0,2054840.0,-81250.0,POINT (2054340.000 -81750.000)
1,2.0,2054340.0,-82250.0,2054840.0,-81750.0,POINT (2054340.000 -82250.000)
2,3.0,2054340.0,-82750.0,2054840.0,-82250.0,POINT (2054340.000 -82750.000)
3,4.0,2054340.0,-83250.0,2054840.0,-82750.0,POINT (2054340.000 -83250.000)
4,5.0,2054340.0,-83750.0,2054840.0,-83250.0,POINT (2054340.000 -83750.000)


In [29]:
grid_latlong.shape

(131610, 9)

In [33]:
grid_latlong["Longitude"] = (grid_latlong["left"] + grid_latlong["right"])/2
grid_latlong["Latitude"] = (grid_latlong["top"] + grid_latlong["bottom"])/2

geometry = [Point(xy) for xy in zip(grid_latlong['Longitude'], grid_latlong['Latitude'])]

grid_latlong["geometry"] = geometry
grid_latlong.head()


,id,left,top,right,bottom,geometry,Longitude,Latitude
0,1.0,2054340.0,-81750.0,2054840.0,-81250.0,POINT (2054590.000 -81500.000),2054590.0,-81500.0
1,2.0,2054340.0,-82250.0,2054840.0,-81750.0,POINT (2054590.000 -82000.000),2054590.0,-82000.0
2,3.0,2054340.0,-82750.0,2054840.0,-82250.0,POINT (2054590.000 -82500.000),2054590.0,-82500.0
3,4.0,2054340.0,-83250.0,2054840.0,-82750.0,POINT (2054590.000 -83000.000),2054590.0,-83000.0
4,5.0,2054340.0,-83750.0,2054840.0,-83250.0,POINT (2054590.000 -83500.000),2054590.0,-83500.0


In [35]:
# Define the Lambert Azimuthal Equal Area projection
projection = pyproj.Proj("+proj=laea +lat_0=45 +lon_0=-100 +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs")

# Define the WGS84 geographic coordinate system
geographic_crs = pyproj.CRS("EPSG:4326")

# Convert the geometry coordinates from meters to degrees
grid_latlong['geometry'] = grid_latlong['geometry'].to_crs(geographic_crs)

grid_latlong.head()

,id,left,top,right,bottom,geometry,Longitude,Latitude
0,1.0,2054340.0,-81750.0,2054840.0,-81250.0,POINT (-74.95440 41.41689),2054590.0,-81500.0
1,2.0,2054340.0,-82250.0,2054840.0,-81750.0,POINT (-74.95615 41.41265),2054590.0,-82000.0
2,3.0,2054340.0,-82750.0,2054840.0,-82250.0,POINT (-74.95790 41.40841),2054590.0,-82500.0
3,4.0,2054340.0,-83250.0,2054840.0,-82750.0,POINT (-74.95964 41.40416),2054590.0,-83000.0
4,5.0,2054340.0,-83750.0,2054840.0,-83250.0,POINT (-74.96139 41.39992),2054590.0,-83500.0


In [37]:
# Extract latitude and longitude from the geometry column
grid_latlong["Center_Latitude"] = grid_latlong["geometry"].y
grid_latlong["Center_Longitude"] = grid_latlong["geometry"].x

grid_latlong = grid_latlong[["Center_Latitude","Center_Longitude"]]
grid_latlong.head()

,Center_Latitude,Center_Longitude
0,41.416894,-74.954401
1,41.412650,-74.956148
2,41.408405,-74.957895
3,41.404161,-74.959642
4,41.399917,-74.961389


In [39]:
#grid_latlong.to_csv("C:/Users/omhai/OneDrive/Desktop/Shetty/Capstone Project/Landcover/Temp/latlong_NJ_degrees.csv")

In [11]:
#check resolution of raster
with rasterio.open("C:/Users/omhai/OneDrive/Desktop/Shetty/Capstone Project/Landcover/NJ_All Layers_4326_500_right_cropped_adjusted.tif") as src:
    # Print the bounding box coordinates
    print('Bounding box coordinates:', src.bounds)

    # Print the CRS
    print('CRS:', src.crs)

    #print resolution of raster
    print('Resolution:', src.res)

Bounding box coordinates: BoundingBox(left=2054340.0, bottom=-349250.0, right=2177340.0, top=-81750.0)
CRS: PROJCS["WGS_1984_Lambert_Azimuthal_Equal_Area",GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]],PROJECTION["Lambert_Azimuthal_Equal_Area"],PARAMETER["latitude_of_center",45],PARAMETER["longitude_of_center",-100],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["metre",1],AXIS["Easting",EAST],AXIS["Northing",NORTH]]
Resolution: (500.0, 500.0)


In [45]:
nightlights_zonal = pd.read_csv("C:/Users/omhai/OneDrive/Desktop/Shetty/Capstone Project/Landcover/Temp/zonal_nightlights_day_01.csv")
nightlights_zonal.head()

,id,left,top,right,bottom,_count,_sum,_mean
0,1,2054340,-81750,2054840,-82250,NaN,NaN,NaN
1,2,2054340,-82250,2054840,-82750,NaN,NaN,NaN
2,3,2054340,-82750,2054840,-83250,NaN,NaN,NaN
3,4,2054340,-83250,2054840,-83750,NaN,NaN,NaN
4,5,2054340,-83750,2054840,-84250,NaN,NaN,NaN


In [46]:
nightlights_mean = nightlights_zonal[["id","_mean"]]
nightlights_mean.head()

,id,_mean
0,1,NaN
1,2,NaN
2,3,NaN
3,4,NaN
4,5,NaN


In [47]:
nightlights_mean['_mean'].notna().sum()

69750

In [48]:
nightlights_mean=nightlights_mean.dropna()
nightlights_mean.shape

(69750, 2)

In [42]:
wind.head()

,ID,Unnamed: 0,latitude,longitude,value,Distance,Vmax,Rmax
0,0,0,41.415450,-75.846241,128,325.294594,30.030877,120.546311
1,1,1,41.410565,-75.846241,128,325.583200,30.100979,120.546311
2,2,2,41.405680,-75.846241,128,325.871899,30.102396,120.546311
3,3,3,41.400794,-75.846241,128,326.160691,30.173096,120.546311
4,4,4,41.395909,-75.846241,128,326.449577,30.174527,120.546311


In [43]:
wind = wind.drop(['Unnamed: 0', 'value','Distance','Rmax'], axis=1)
wind.head()

,ID,latitude,longitude,Vmax
0,0,41.415450,-75.846241,30.030877
1,1,41.410565,-75.846241,30.100979
2,2,41.405680,-75.846241,30.102396
3,3,41.400794,-75.846241,30.173096
4,4,41.395909,-75.846241,30.174527


In [49]:
regression_df = landcover_zonal.merge(wind,how="left", left_on="id",right_on="ID").fillna(0)
regression_df = regression_df.merge(nightlights_mean,how="inner",on="id")
regression_df.head()

,id,left,top,right,bottom,HISTO_0,HISTO_1,HISTO_5,HISTO_6,HISTO_8,HISTO_10,HISTO_14,HISTO_15,HISTO_16,HISTO_17,HISTO_18,HISTO_NODATA,ID,latitude,longitude,Vmax,_mean
0,99,2054340,-130750,2054840,-131250,0.0,0.0,0.346021,3.114187,0.0,0.0,0.0,0.0,0.0,0.346021,13.840830,82.352941,99.0,40.931814,-75.846241,34.452763,0.800000
1,100,2054340,-131250,2054840,-131750,0.0,0.0,41.522491,4.152249,0.0,0.0,0.0,0.0,0.0,8.304498,22.491349,23.529412,100.0,40.926929,-75.846241,34.455151,0.843006
2,101,2054340,-131750,2054840,-132250,0.0,0.0,67.279412,0.000000,0.0,0.0,0.0,0.0,0.0,7.352941,13.602941,11.764706,101.0,40.922044,-75.846241,34.457554,0.900000
3,102,2054340,-132250,2054840,-132750,0.0,0.0,39.446367,0.000000,0.0,0.0,0.0,0.0,0.0,13.148789,13.494810,33.910035,102.0,40.917158,-75.846241,34.459970,1.244677
4,103,2054340,-132750,2054840,-133250,0.0,0.0,31.833910,7.958478,0.0,0.0,0.0,0.0,0.0,13.494810,12.802768,33.910035,103.0,40.912273,-75.846241,34.572871,1.050000


In [50]:
regression_df.shape

(69750, 22)

In [51]:
regression_df.columns

Index(['id', 'left', 'top', 'right', 'bottom', 'HISTO_0', 'HISTO_1', 'HISTO_5',
       'HISTO_6', 'HISTO_8', 'HISTO_10', 'HISTO_14', 'HISTO_15', 'HISTO_16',
       'HISTO_17', 'HISTO_18', 'HISTO_NODATA', 'ID', 'latitude', 'longitude',
       'Vmax', '_mean'],
      dtype='object')

In [53]:
regression_df = regression_df.drop(['id', 'left', 'top', 'right', 'bottom','ID', 'latitude',
       'longitude'], axis=1)
regression_df.shape

(69750, 14)

In [54]:
regression_df.columns

Index(['HISTO_0', 'HISTO_1', 'HISTO_5', 'HISTO_6', 'HISTO_8', 'HISTO_10',
       'HISTO_14', 'HISTO_15', 'HISTO_16', 'HISTO_17', 'HISTO_18',
       'HISTO_NODATA', 'Vmax', '_mean'],
      dtype='object')

- percentage values of landcover
- Do regression again 
- drop distance (from hurricane) and rmax from wind
- input from before day (1st day or can A3 data) and output(hurricane day)
- replicate the same for hurricane michael and ian (get coordinates for wind)
- latitude and longitude coordinates required for wind

In [56]:
df = regression_df.copy()

df = df.drop("HISTO_NODATA", axis=1)

# Separate independent and dependent variables
X = df.drop('_mean', axis=1)  # Independent variables
y = df['_mean']  # Dependent variable

# Standardize independent variables
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Create and fit the regression model
reg_model = LinearRegression()
reg_model.fit(X_train, y_train)

# Print the model coefficients
print("Model Coefficients:")
for feature, coef in zip(X.columns, reg_model.coef_):
    print(feature, ":", coef)

# Predict on the test set
y_pred = reg_model.predict(X_test)

# Calculate R2 score on the test set
r2 = r2_score(y_test, y_pred)

# Print the R2 score
print("R2 score on the test set:", r2)

Model Coefficients:
HISTO_0 : -0.29726569722653035
HISTO_1 : -1.4689373706650686
HISTO_5 : -2.9130858382800655
HISTO_6 : -1.4107869380102145
HISTO_8 : -0.3904659444128858
HISTO_10 : -0.2375465142071818
HISTO_14 : -2.090742299139166
HISTO_15 : -2.6181427402549957
HISTO_16 : -0.2925025911068274
HISTO_17 : 5.462393873715373
HISTO_18 : -0.2541421419943326
Vmax : 0.3038252040332421
R2 score on the test set: 0.3162650138460211
